Downloading Dependencies

In [30]:
# %pip install -U llama-index llama-parse llama-hub
# %pip install llama-index-vector-stores-redis
# %pip install llama-index-storage-chat-store-redis
# %pip install llama-index-embeddings-cohore
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-storage-docstore-redis
# %pip install llama-index-storage-index-store-redis

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

Setup and Download Data

In [19]:
# sudo apt install lsb-release curl gpg
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

# sudo apt-get update
# !sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes

/bin/bash: line 1: redis-stack-server: command not found


In [1]:
REDIS_HOST = "127.0.0.1"
REDIS_PORT = 6379
REDIS_PASSWORD = ""


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv("Llama_Parser_Key")
os.environ["CO_API_KEY"] = "YOUR API KEY"

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf' -O 'data/2022-chevrolet-colorado-ebrochure.pdf'


--2024-05-15 10:42:16--  https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3566101 (3.4M) [application/octet-stream]
Saving to: ‘data/2022-chevrolet-colorado-ebrochure.pdf’

data/2022-chevrolet 100%[===================>]   3.40M  17.3MB/s    in 0.2s    

2024-05-15 10:42:17 (17.3 MB/s) - ‘data/2022-chevrolet-colorado-ebrochure.pdf’ saved [3566101/3566101]



In [5]:
from llama_parse import LlamaParse

from llama_index.core import SimpleDirectoryReader

parser = LlamaParse(
    result_type= "markdown"
)


/home/jellyfish/llms/simienv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
file_extractor = {".pdf":parser}

In [8]:
reader = SimpleDirectoryReader("./data",file_extractor=file_extractor)

In [9]:
documents = reader.load_data()

Started parsing the file under job_id e8e50791-a5ce-41ca-8760-4945a5af8ffc


In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import DocstoreStrategy, IngestionPipeline, IngestionCache
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore
from redisvl.schema import IndexSchema

In [12]:
embed_model = HuggingFaceEmbedding(text_instruction = "search_document")

In [13]:
custom_schema = IndexSchema.from_dict(
    {
        "index":{
            "name":"chevy-colorado",
            "prefix":"pdf:chunk",
            "key_separator": ":"
        },
        "fields":[
            {"type":"tag","name":"id"},
            {"type":"tag","name":"doc_id"},
            {"type":"text","name":"text"},
            {
                "type":"vector",
                "name": "vector",
                "attrs":{
                    "dims":1024,
                    "algorithm":"hnsw",
                    "distance_metric":"cosine"
                },
            },
        ]
    }
)

In [14]:
custom_schema

IndexSchema(index=IndexInfo(name='chevy-colorado', prefix='pdf:chunk', key_separator=':', storage_type=<StorageType.HASH: 'hash'>), fields={'id': TagField(name='id', type='tag', path=None, attrs=TagFieldAttributes(sortable=False, separator=',', case_sensitive=False, withsuffixtrie=False)), 'doc_id': TagField(name='doc_id', type='tag', path=None, attrs=TagFieldAttributes(sortable=False, separator=',', case_sensitive=False, withsuffixtrie=False)), 'text': TextField(name='text', type='text', path=None, attrs=TextFieldAttributes(sortable=False, weight=1, no_stem=False, withsuffixtrie=False, phonetic_matcher=None)), 'vector': HNSWVectorField(name='vector', type='vector', path=None, attrs=HNSWVectorFieldAttributes(dims=1024, algorithm=<VectorIndexAlgorithm.HNSW: 'HNSW'>, datatype=<VectorDataType.FLOAT32: 'FLOAT32'>, distance_metric=<VectorDistanceMetric.COSINE: 'COSINE'>, initial_cap=None, m=16, ef_construction=200, ef_runtime=10, epsilon=0.01))}, version='0.1.0')

In [32]:
vector_index_pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        embed_model,
    ],
    docstore= RedisDocumentStore.from_host_and_port(
        REDIS_HOST, REDIS_PORT, namespace = "doc-store"
    ),
    vector_store= RedisVectorStore(
        schema = custom_schema,
        redis_url = f"redis://{REDIS_HOST}:{REDIS_PORT}"
    ),
    cache= IngestionCache(
        cache= RedisCache.from_host_and_port(REDIS_HOST, REDIS_PORT),
        collection= "doc-cache"
    ),
    docstore_strategy= DocstoreStrategy.UPSERTS,
)

ValueError: Required Redis database module searchlight with version >= 20600 not installed. Refer to Redis Stack documentation: https://redis.io/docs/stack/